In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
import keras

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from torch.utils.data import DataLoader,TensorDataset

In [ ]:
def dataloader():
    data = []
    for i in range(1918):
        X = np.load('D:\\实验\\实验数据\\吸收谱-cnn\\{}.npy'.format(i + 1))
        data.append(X)
    print('数据长度是 {}'.format(len(data)))
    a = [0 for i in range(694)]  # 第一个样本的数量A
    b = [1 for i in range(634)]  # 第二个样本的数量B
    c = [2 for i in range(590)]  # 第三个样本的数量Normal
    a = np.concatenate((a, b, c), axis=0)
    labels = keras.utils.to_categorical(a)  # 生成标签
    return data,labels,a

In [ ]:
def dataloader():
    data = []
    for i in range(2625):
        X = np.load('D:\实验\实验数据\高光谱-cnn\{}.npy'.format(i + 1))
        data.append(X)
    print('数据长度是 {}'.format(len(data)))
    a = [0 for i in range(895)]  # 第一个样本的数量A
    b = [1 for i in range(819)]  # 第二个样本的数量B
    c = [2 for i in range(911)]  # 第三个样本的数量Normal
    a = np.concatenate((a, b, c), axis=0)
    labels = keras.utils.to_categorical(a)  # 生成标签
    return data,labels,a

In [ ]:
def thzlabelloder():
    a = [0 for i in range(698)]
    b = [1 for i in range(656)]  # 第二个样本的数量B
    c = [2 for i in range(626)]  # 第三个样本的数量Normal
    a = np.concatenate((a, b, c), axis=0)
    labels = keras.utils.to_categorical(a)
    return labels

In [ ]:
def ggplabelloder():
    a = [0 for i in range(896)]
    b = [1 for i in range(855)]  # 第二个样本的数量B
    c = [2 for i in range(913)]  # 第三个样本的数量Normal
    a = np.concatenate((a, b, c), axis=0)
    labels = keras.utils.to_categorical(a)
    return labels

In [ ]:
data, labels, y = dataloader() 

In [ ]:
data = pd.read_excel('baseline.xlsx', header=None).values
labels = thzlabelloder()

In [ ]:
labels.shape

In [ ]:
data = pd.read_excel('ggp.xlsx', header=None).values
labels = ggplabelloder()

In [ ]:
data = data[1:,:]

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(data, labels, test_size=0.3)

In [ ]:
train_x = np.array(train_x)

In [ ]:
valid_x = np.array(valid_x)

In [ ]:
train_x = train_x[:,:,:,17:117]

In [ ]:
valid_x = valid_x[:,:,:,17:117]

In [ ]:
train_x = train_x[:,:,:,15:215]

In [ ]:
valid_x = valid_x[:,:,:,15:215]

In [ ]:
train_mean = np.mean(train_x, axis=0)
train_std = np.std(train_x, axis=0)
valid_mean = np.mean(valid_x, axis=0)
valid_std = np.std(valid_x, axis=0)

In [ ]:
train_std.shape

In [ ]:
train_x = (train_x - train_mean) / train_std
valid_x = (valid_x - valid_mean) / valid_std

In [ ]:
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],1)
valid_x = valid_x.reshape(valid_x.shape[0],valid_x.shape[1],1)

In [ ]:
train_x.shape

In [ ]:
valid_x.shape

In [ ]:
from keras.layers import Dense, Activation,Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization,regularizers,AveragePooling2D,Conv1D
from keras.layers import MaxPooling1D,AveragePooling1D,LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
#平均高光谱处理
model = Sequential()
model.add(Conv1D(filters = 128, kernel_size = 2,input_shape = (94,1), kernel_regularizer=regularizers.l2(0.01) ,activation = 'relu'))
model.add(Conv1D(filters = 64, kernel_size = 2, kernel_regularizer=regularizers.l2(0.01), activation = 'relu'))
model.add(AveragePooling1D(pool_size = 2))
model.add(Flatten())
model.add(Dense(units = 256))
model.add(Dense(units = 3))
model.add(Activation('softmax'))

In [ ]:
#太赫兹吸收谱平均
model = Sequential()
model.add(Conv1D(filters = 128, kernel_size = 2,input_shape = (94,1), kernel_regularizer=regularizers.l2(0.01),activation = 'relu'))
model.add(Conv1D(filters = 64, kernel_size = 2, kernel_regularizer=regularizers.l2(0.01), activation = 'relu'))
model.add(AveragePooling1D(pool_size = 2))
model.add(Flatten())
model.add(Dense(units = 256))
model.add(Dense(units = 3))
model.add(Activation('softmax'))

In [ ]:
#高光谱和太赫兹吸收图像
model = Sequential()
model.add(Conv2D(filters = 128, kernel_size = 2, input_shape = (13,4,200), kernel_regularizer=regularizers.l2(0.05),activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 2, kernel_regularizer=regularizers.l2(0.05),activation = 'relu'))
model.add(AveragePooling2D(pool_size = 2))
model.add(Flatten())
model.add(Dense(units = 64))
model.add(Dense(units = 3))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=['accuracy'])  

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

In [ ]:
history=LossHistory()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 5000, mode='auto', verbose = 1)

In [ ]:
tensorboard = TensorBoard(log_dir='./20190731THz2D', histogram_freq=100, batch_size=128, write_graph=True, write_grads=False, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch' )

In [ ]:
history=model.fit(train_x, train_y, epochs=20,batch_size=256, validation_data=(valid_x, valid_y), callbacks=[history])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("model loss and acc")
plt.ylabel("loss and acc") 
plt.xlabel("epoch")
plt.legend(["train","test","train_acc","test_acc"],loc="upper right")
plt.savefig("1.jpg")
plt.show()

In [ ]:
model.save('20190802ggp2D.h5')

# 提取步骤

In [ ]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import cv2
import os 
import matplotlib.pyplot as plt

In [ ]:
result = np.argmax(valid_y,axis=1)

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('ggp1D-output.xlsx')
worksheet=workbook.add_worksheet()
hang = result.shape[0]
for col in range(int(hang)):
        worksheet.write(col,0,result[col,])
workbook.close()

In [ ]:
x = valid_x
x = np.sum(x,-1)
x.shape

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('thz1D-output.xlsx')
worksheet=workbook.add_worksheet()
hang = x.shape[0]
lie = x.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,x[col,row])
workbook.close()

In [ ]:
result = np.argmax(output,axis=1)

In [ ]:
from keras.models import load_model
model = load_model('20190802ggp2D.h5')

In [ ]:
model.summary()

In [ ]:
from keras.models import Model

In [ ]:
layer_name = 'conv2d_1'
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)#创建的新模型
output = intermediate_layer_model.predict(valid_x)

In [ ]:
output.shape

In [ ]:
valid_x.shape

In [ ]:
a1 = np.array(valid_x)

In [ ]:
a1 = np.array(output)

In [ ]:
a1 = a1.reshape(a1.shape[0],1,a1.shape[1])

In [ ]:
a1 = np.sum(a1,-1)

In [ ]:
a1 = np.sum(a1,0)

In [ ]:
a1.shape

In [ ]:
pic = a1[2,:,:,1]

In [ ]:
pic.shape

In [ ]:
plt.imshow(pic)

In [ ]:
def save_seed_img(data):
    for j in range(100):
        t = data[2,:,:,j]
        plt.imshow(t)
        list_i = str(j)
        if '1test1' not in os.listdir('./'):
            os.makedirs('1test1')
        plt.savefig('./1test1/' + '_'.join(list_i) + '.jpg',dpi = 1000)
        plt.show()

In [ ]:
save_seed_img(a1)

In [ ]:
valid_x.shape

In [ ]:
aa = np.sum(valid_x,-1)

In [ ]:
aa.shape

In [ ]:
plt.plot(a1[1,:])

In [ ]:
plt.imshow(aa[6,:,:])

In [ ]:
def save_seed_img(data):
    for j in range(100):
        t = data[j,:]
        plt.plot(t)
        list_i = str(j)
        if 'ggp1D-dense2' not in os.listdir('./'):
            os.makedirs('ggp1D-dense2')
        plt.savefig('./ggp1D-dense2/' + '_'.join(list_i) + '.jpg',dpi = 1000)
        plt.show()

In [ ]:
def save_seed_img(data):
        plt.plot(data)
        list_i = str(111)
        if 'ggp1D-conv1' not in os.listdir('./'):
            os.makedirs('ggp1D-conv1')
        plt.savefig('./ggp1D-conv1/' + '_'.join(list_i) + '.jpg',dpi = 1000)
        plt.show()

In [ ]:
save_seed_img(a1)

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('ggp1D-dense2.xlsx')
worksheet=workbook.add_worksheet()
hang = a1.shape[0]
lie = a1.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,a1[col,row])
workbook.close()

# 权重提取

# weigth提取

In [ ]:
layer_name = 'dense_10'
w1 = model.get_layer(layer_name).get_weights()

In [ ]:
w1 = np.array(w1)

In [ ]:
w1.shape

In [ ]:
w1 = w1[0]

In [ ]:
w1.shape

In [ ]:
a1 = w1

In [ ]:
a1 = np.sum(a1,-1)

In [ ]:
a1.shape

In [ ]:
a1 = np.sum(a1,1)

In [ ]:
a1 = np.sum(a1,0)

In [ ]:
a1.shape

In [ ]:
plt.plot(a1)

In [ ]:
def save_seed_img(data):
        plt.plot(data)
        list_i = str(4)
        if 'ggp1D-dense2-weight' not in os.listdir('./'):
            os.makedirs('ggp1D-dense2-weight')
        plt.savefig('./ggp1D-dense2-weight/' + '_'.join(list_i) + '.jpg',dpi = 1000)
        plt.show()

In [ ]:
save_seed_img(a1)

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('ggp1D-dense2-weight.xlsx')
worksheet=workbook.add_worksheet()
hang = a1.shape[0]
lie = a1.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,a1[col,row])
workbook.close()

# bias提取

In [ ]:
w2 = w1[1]

In [ ]:
w2.shape

In [ ]:
a2= w2

In [ ]:
a2 = np.sum(a2,-1)

In [ ]:
a2.shape

In [ ]:
a2 = np.sum(a2,0)

In [ ]:
plt.plot(a2)

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('ggp1D-dense1-bias.xlsx')
worksheet=workbook.add_worksheet()
hang = a2.shape[0]
lie = a2.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,a2[col,row])
workbook.close()

In [ ]:
import xlsxwriter
workbook=xlsxwriter.Workbook('ggp1D-dense2-bias.xlsx')
worksheet=workbook.add_worksheet()
hang = a2.shape[0]
for col in range(int(hang)):
        worksheet.write(col,0,a2[col,])
workbook.close()

In [ ]:
def save_seed_img(data):
        plt.plot(data)
        list_i = str(1)
        if 'ggp1D-dense2-bias' not in os.listdir('./'):
            os.makedirs('ggp1D-dense2-bias')
        plt.savefig('./ggp1D-dense2-bias/' + '_'.join(list_i) + '.jpg',dpi = 1000)
        plt.show()

In [ ]:
save_seed_img(a2)

train_x1 = pd.read_excel('1.35m-平均光谱-715.xlsx', header=None).values
train_mean1 = np.mean(train_x1, axis=0).reshape(1, -1)
train_std1 = np.std(train_x1, axis=0).reshape(1, -1)
train_x1 = (train_x1 - train_mean1) / train_std1
train_x1 = train_x1.reshape(train_x1.shape[0],train_x1.shape[1],1)

train_x2 = pd.read_excel('25F-平均光谱-700.xlsx', header=None).values
train_mean2 = np.mean(train_x2, axis=0).reshape(1, -1)
train_std2 = np.std(train_x2, axis=0).reshape(1, -1)
train_x2 = (train_x2 - train_mean2) / train_std2
train_x2 = train_x2.reshape(train_x2.shape[0],train_x2.shape[1],1)

train_x3 = pd.read_excel('光1-平均光谱-602.xlsx', header=None).values
train_mean3 = np.mean(train_x3, axis=0).reshape(1, -1)
train_std3 = np.std(train_x3, axis=0).reshape(1, -1)
train_x3 = (train_x3 - train_mean3) / train_std3
train_x3 = train_x3.reshape(train_x3.shape[0],train_x3.shape[1],1)

train_x4 = pd.read_excel('光F-平均光谱-712.xlsx', header=None).values
train_mean4 = np.mean(train_x4, axis=0).reshape(1, -1)
train_std4 = np.std(train_x4, axis=0).reshape(1, -1)
train_x4 = (train_x4 - train_mean1) / train_std4
train_x4 = train_x4.reshape(train_x4.shape[0],train_x4.shape[1],1)

train_x5 = pd.read_excel('光m-平均光谱-616.xlsx', header=None).values
train_mean5 = np.mean(train_x5, axis=0).reshape(1, -1)
train_std5 = np.std(train_x5, axis=0).reshape(1, -1)
train_x5 = (train_x5 - train_mean5) / train_std5
train_x5 = train_x5.reshape(train_x5.shape[0],train_x5.shape[1],1)

train_x6 = pd.read_excel('浙丝35-平均光谱-783.xlsx', header=None).values
train_mean6 = np.mean(train_x6, axis=0).reshape(1, -1)
train_std6 = np.std(train_x6, axis=0).reshape(1, -1)
train_x6 = (train_x6 - train_mean6) / train_std6
train_x6 = train_x6.reshape(train_x6.shape[0],train_x6.shape[1],1)

In [ ]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import cv2
import os 
import matplotlib.pyplot as plt

In [ ]:
def cal_pairwise_dist(x):
    '''计算pairwise 距离, x是matrix
    (a-b)^2 = a^w + b^2 - 2*a*b
    '''
    sum_x = np.sum(np.square(x), 1)
    dist = np.add(np.add(-2 * np.dot(x, x.T), sum_x).T, sum_x)
    return dist
def cal_perplexity(dist, idx=0, beta=1.0):
    '''计算perplexity, D是距离向量，
    idx指dist中自己与自己距离的位置，beta是高斯分布参数
    这里的perp仅计算了熵，方便计算
    '''
    prob = np.exp(-dist * beta)
    # 设置自身prob为0
    prob[idx] = 0
    sum_prob = np.sum(prob)
    perp = np.log(sum_prob) + beta * np.sum(dist * prob) / sum_prob
    prob /= sum_prob
    return perp, prob
def seach_prob(x, tol=1e-5, perplexity=30.0):
    '''二分搜索寻找beta,并计算pairwise的prob
    '''
 
    # 初始化参数
    print("Computing pairwise distances...")
    (n, d) = x.shape
    dist = cal_pairwise_dist(x)
    pair_prob = np.zeros((n, n))
    beta = np.ones((n, 1))
    # 取log，方便后续计算
    base_perp = np.log(perplexity)
 
    for i in range(n):
        if i % 500 == 0:
            print("Computing pair_prob for point %s of %s ..." %(i,n))
 
        betamin = -np.inf
        betamax = np.inf
        perp, this_prob = cal_perplexity(dist[i], i, beta[i])
 
        # 二分搜索,寻找最佳sigma下的prob
        perp_diff = perp - base_perp
        tries = 0
        while np.abs(perp_diff) > tol and tries < 50:
            if perp_diff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2
                else:
                    beta[i] = (beta[i] + betamin) / 2
 
            # 更新perb,prob值
            perp, this_prob = cal_perplexity(dist[i], i, beta[i])
            perp_diff = perp - base_perp
            tries = tries + 1
        # 记录prob值
        pair_prob[i,] = this_prob
    print("Mean value of sigma: ", np.mean(np.sqrt(1 / beta)))
    return pair_prob
def pca(x, no_dims):
    ''' PCA算法
    使用PCA先进行预降维
    '''
    print("Preprocessing the data using PCA...")
    (n, d) = x.shape
    x = x - np.tile(np.mean(x, 0), (n, 1))
    l, M = np.linalg.eig(np.dot(x.T, x))
    y = np.dot(x, M[:,0:no_dims])
    return y
def tsne(x, no_dims, initial_dims, perplexity, max_iter):
    """Runs t-SNE on the dataset in the NxD array x
    to reduce its dimensionality to no_dims dimensions.
    The syntaxis of the function is Y = tsne.tsne(x, no_dims, perplexity),
    where x is an NxD NumPy array.
    """
 
    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array x should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1
 
    # 初始化参数和变量
    x = pca(x, initial_dims).real
    (n, d) = x.shape
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 500
    min_gain = 0.01
    y = np.random.randn(n, no_dims)
    dy = np.zeros((n, no_dims))
    iy = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))
 
    # 对称化
    P = seach_prob(x, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    # early exaggeration
    P = P * 4
    P = np.maximum(P, 1e-12)
 
    # Run iterations
    for iter in range(max_iter):
        # Compute pairwise affinities
        sum_y = np.sum(np.square(y), 1)
        num = 1 / (1 + np.add(np.add(-2 * np.dot(y, y.T), sum_y).T, sum_y))
        num[range(n), range(n)] = 0
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)
 
        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dy[i,:] = np.sum(np.tile(PQ[:,i] * num[:,i], (no_dims, 1)).T * (y[i,:] - y), 0)
 
        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dy > 0) != (iy > 0)) + (gains * 0.8) * ((dy > 0) == (iy > 0))
        gains[gains < min_gain] = min_gain
        iy = momentum * iy - eta * (gains * dy)
        y = y + iy
        y = y - np.tile(np.mean(y, 0), (n, 1))
        # Compute current value of cost function
        if (iter + 1) % 100 == 0:
            if iter > 100:
                C = np.sum(P * np.log(P / Q))
            else:
                C = np.sum( P/4 * np.log( P/4 / Q))
            print("Iteration ", (iter + 1), ": error is ", C)
        # Stop lying about P-values
        if iter == 100:
            P = P / 4
    print("finished training!")
    return y

In [ ]:
if __name__ == "__main__":
    # Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.
    X1 = tt
    X2 = intermediate_output
    Y = tsne(X1, 2, 200, 30.0, 500)
    Y2 = tsne(X2, 2, 6, 30.0, 500)

In [ ]:
Y2

In [ ]:
    labels = pd.read_excel('label.xlsx', header=None).values
    from matplotlib import pyplot as plt
    for i in range(len(labels)):
        if labels[i] == 1:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1 ,1],2,color='red')
        if labels[i] == 2:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1, 1],2,color='green')
        if labels[i] == 3:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1 ,1],2,color='blue')
        if labels[i] == 4:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1, 1],2,color='black')
        if labels[i] == 5:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1, 1],2,color='yellow')
        if labels[i] == 6:
            plt.scatter(Y[i:i+1, 0], Y[i:i+1, 1],2,color='cyan')
    plt.savefig("纹理秋葵6类之前.jpg",dpi = 1000)
    plt.show()

In [ ]:
    for i in range(len(labels)):
        if labels[i] == 1:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1 ,1],2,color='red')
        if labels[i] == 2:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1, 1],2,color='green')
        if labels[i] == 3:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1 ,1],2,color='blue')
        if labels[i] == 4:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1, 1],2,color='black')
        if labels[i] == 5:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1, 1],2,color='yellow')
        if labels[i] == 6:
            plt.scatter(Y2[i:i+1, 0], Y2[i:i+1, 1],2,color='cyan')
    plt.savefig("纹理秋葵6类之后.jpg",dpi = 1000)
    plt.show()

test_x = pd.read_excel('预测数据.xlsx', header=None).values
test_y = pd.read_excel('预测标签.xlsx', header=None).values
test_x = ((test_x - train_mean) / train_std).reshape(test_x.shape[0],1005,1)

model.evaluate(test_x, test_y)

shuchu = model.layers[19].output

shuchu

predict_test=model.predict(test_x)

np.argmax(predict_test,axis=1)

In [ ]:
import xlsxwriter
data_x = pd.read_excel('1227-x.xlsx', header=None).values
data_y = pd.read_excel('1227-y.xlsx', header=None).values
train_x, valid_x, train_y, valid_y = train_test_split(data_x, data_y, test_size=0.3)

In [ ]:
workbook=xlsxwriter.Workbook('1227-5880-x.xlsx')
worksheet=workbook.add_worksheet()
hang = train_x.shape[0]
lie = train_x.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,train_x[col,row])
workbook.close()

In [ ]:
workbook=xlsxwriter.Workbook('1227-2520-x.xlsx')
worksheet=workbook.add_worksheet()
hang = valid_x.shape[0]
lie = valid_x.shape[1]
for col in range(int(hang)):
    for row in range(int(lie)):
        worksheet.write(col,row,valid_x[col,row])
workbook.close()

In [ ]:
test1_x = pd.read_excel('testx.xlsx', header=None).values
train_mean = np.mean(test1_x, axis=0).reshape(1, -1)
train_std = np.std(test1_x, axis=0).reshape(1, -1)
predict_test = ((test1_x - train_mean) / train_std).reshape(test1_x.shape[0],2151,1)
predict_test=model.predict(predict_test)
predict_test= np.argmax(predict_test,axis=1)
workbook=xlsxwriter.Workbook('test-y.xlsx')
worksheet=workbook.add_worksheet()
hang = predict_test.shape[0]
for col in range(int(hang)):
    worksheet.write(col,0,predict_test[col])
workbook.close()

In [ ]:
test1_x = pd.read_excel('1227-2520-x.xlsx', header=None).values
train_mean = np.mean(test1_x, axis=0).reshape(1, -1)
train_std = np.std(test1_x, axis=0).reshape(1, -1)
predict_test = ((test1_x - train_mean) / train_std).reshape(test1_x.shape[0],417,1)
predict_test=model.predict(predict_test)
predict_test= np.argmax(predict_test,axis=1)
workbook=xlsxwriter.Workbook('1227-2520-label.xlsx')
worksheet=workbook.add_worksheet()
hang = predict_test.shape[0]
lie = predict_test.shape[1]
for col in range(int(hang)):
        worksheet.write(col,row,predict_test[col,row])
workbook.close()